<H2 align = 'center'> Student Reviews: Word Count </H2>

<H3> Given a text file containing student review for a course, write a program to count the number of words in the reviews. </H3>
<ol> <li> Exclude Boring Words listed in /data/trendytech/boringwords.txt </li> 
     <li> Note that some words may appear in different case; treat them as the same </li>
</ol>


In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("RDDAssignment3").getOrCreate()

In [3]:
student_reviews = spark.sparkContext.textFile("/public/trendytech/reviews/trendytech-student-reviews.csv")

In [38]:
student_review_all_words = student_reviews.flatMap(lambda x:x.split(" ")).map(lambda x: x.lower())

In [39]:
student_review_all_words.take(10)

['i',
 'got',
 'to',
 'know',
 'about',
 'this',
 'course',
 'by',
 'recommendation',
 'from']

In [40]:
student_review_all_words.count()

9014

In [41]:
! hadoop fs -put /data/trendytech/boringwords.txt /user/itv012857/data/

put: `/user/itv012857/data/boringwords.txt': File exists


In [42]:
boringwords_rdd = spark.sparkContext.textFile("/user/itv012857/data/boringwords.txt")

In [43]:
boringwords_rdd.count()

10461

In [44]:
boring_words_broadcast = spark.sparkContext.broadcast(boringwords_rdd.collect())

In [45]:
boring_words_broadcast.value[0:5]

['shouldnt', 'worrying', 'simplify', 'tidy', 'shouldnt']

In [46]:
filtered_words = student_review_all_words.filter(lambda x: x not in boring_words_broadcast.value)

In [47]:
filtered_words.take(5)

['lectures(content)', 'explored', 'mind.', 'sumit', 'passionate']

In [48]:
filtered_words_with_count = filtered_words.map(lambda x : (x,1))

In [49]:
filtered_words_with_count.take(10)

[('lectures(content)', 1),
 ('explored', 1),
 ('mind.', 1),
 ('sumit', 1),
 ('passionate', 1),
 ('data', 1),
 ('content.', 1),
 ('wonders', 1),
 ('.i', 1),
 ('data', 1)]

In [50]:
word_count = filtered_words_with_count.reduceByKey(lambda x,y: x+y)

In [51]:
word_count.take(10)

[('lectures(content)', 1),
 ('explored', 1),
 ('sumit', 109),
 ('passionate', 6),
 ('wonders', 1),
 ('domain.', 12),
 ('benifitted', 1),
 ('career.', 7),
 ('', 64),
 ('trendytech', 67)]

In [52]:
word_count.sortBy(lambda x: x[1], ascending = False).take(20)

[('data', 201),
 ('sumit', 109),
 ('trendytech', 67),
 ('', 64),
 ('data.', 34),
 ('course.', 33),
 ("sir's", 23),
 ('trendy', 14),
 ('course,', 13),
 ("master's", 13),
 ('domain.', 12),
 ("trendytech's", 12),
 ('sir.', 11),
 ('program.', 9),
 ('field.', 9),
 ('concepts.', 9),
 ('hands-on', 8),
 ('fresher', 8),
 ('amazing.', 8),
 ('career.', 7)]